In [2]:
from glob import glob
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import orjson as json
import random

In [ ]:
data = []


with open('facebook.jsonl') as fopen:
    for x in tqdm(fopen):
        try:
            data.append(json.loads(x))
        except:
            pass

120770it [01:34, 1527.55it/s]

In [7]:
len(data)

191248

In [8]:
vectors, texts = [], []

for d in data:
    vectors.append(d['v'])
    texts.append(d['text'])

In [12]:
len(vectors)

191248

In [16]:
# !pip3 install pyfnntw

In [25]:
# !pip3 install scipy scikit-learn

In [26]:
from scipy.spatial import KDTree
import sklearn.neighbors
import numpy as np

In [20]:
concat = np.array(vectors)
concat.shape

(191248, 1536)

In [21]:
%%time

kd_tree = KDTree(concat, leafsize = 40)

CPU times: user 2.68 s, sys: 13.7 ms, total: 2.7 s
Wall time: 2.7 s


In [29]:
%%time

sk_kd_tree = sklearn.neighbors.KDTree(concat, leaf_size = 20)

CPU times: user 49.8 s, sys: 88.3 ms, total: 49.9 s
Wall time: 49.6 s


In [118]:
%%time

dist, ind = kd_tree.query(concat[0], k=len(concat))

CPU times: user 228 ms, sys: 47.5 ms, total: 276 ms
Wall time: 275 ms


In [119]:
# import seaborn as sns
# sns.distplot(dist)

In [120]:
dist[dist < 0.7]

array([0.        , 0.64757418, 0.65585836, 0.66123705])

In [123]:
pos_indices = [k for k in ind[dist < 0.7]]
neg_indices = [k for k in ind[dist > 1.3]]
pos_indices

[0, 55471, 44240, 69514]

In [122]:
pos = [texts[i] for i in pos_indices]
pos

['Sy suka Liwat', 'Liwat Liwat Liwat', 'Sokong liwat', 'Liwat sedap ke']

In [99]:
texts[1]

'A Rusmawi A Manafi'

In [97]:
pos = [texts[i] for i in neg_indices]
pos

["Kalau rasa susah nak pergi lhdn, cuba email them. Check email customer care officer of that branch, on lhdn's website.",
 'Compound has to be paid in cash, right? Dont pay if u want to appeal. Appeal first . Bcos compound is paid to the court, thru lhdn. Just like saman trafik. Once paid, u will not get refund. Appeal to the dept that issued u the saman. State ur reasons for late/non compliance to pcb rules',
 'Iron Skull kesian ya. Masih belum boleh move on.\nBesok aku post, pasti ko selit juga. Aku pun tak nak buang masa dgn komen2 bongok ko. Spt aku ckp, kenal aku pun tidak tapi otak ko fikir mcm2 semua merapu.',
 'Ann Wong tapi barang recycle tu selama ni drp pemimpin atasan sampai ahli sendiri yang bagi bad review.. Hoy hoy ya hoy',
 'Cuba call pegawai yg uruskan minta buat Pengurangan..sa\nya pon kene 2015-2021..32K.\n.tapi sy call pegawai, hntr semua dokumen jadi 800+- kene byr',
 'saya pernah walkin ke lhdn sbb nk declare utk income ,bila kata xkerja & hanya jual brg yg dpt k

In [113]:
import pickle
import os

class Pointer:
    def __init__(self, filename):
        self.filename = filename
        self.index = -1

    def _save(self):
        with open(self.filename, 'wb') as fopen:
            pickle.dump(self.index, fopen)

    def increment(self):
        self.index += 1
        self._save()

    def load(self):
        if not os.path.exists(self.filename):
            return
        with open(self.filename, 'rb') as fopen:
            self.index = pickle.load(fopen)

In [114]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py
import mp

def dedup(strings):
    unique_neg = []
    elements = set()

    for n in strings:
        x_lower = n.lower()
        if x_lower not in elements:
            elements.add(x_lower)
            unique_neg.append(n)
    return unique_neg

dedup(['a', 'a'])

['a']

In [137]:
!rm -rf facebook-hard
!mkdir facebook-hard

In [138]:
lower_bound = 0.7
upper_bound = 1.3

def loop(data):
    data, index = data
    filename = f'facebook-hard/{index}.jsonl'
    fopen = open(filename, 'a')
    pointer = Pointer(f'{filename}.pickle')
    pointer.load()
    for n in tqdm(range(len(data))):
        x = data[n]
        if n > pointer.index:
            dist, ind = kd_tree.query(concat[x], k=len(concat))

            query = texts[x]

            pos_indices = [k for k in ind[dist < lower_bound]]
            neg_indices = [k for k in ind[dist > upper_bound]]
            
            if len(pos_indices) > 6:
                pos_indices = random.sample(pos_indices,6)
            if len(neg_indices) > 5:
                neg_indices = random.sample(neg_indices,5)

            pos = [texts[i] for i in pos_indices if texts[i] != query and len(texts[i]) > 1]
            pos = dedup(pos)

            if len(pos) == 0:
                continue

            neg = [texts[i] for i in neg_indices if texts[i] != query and len(texts[i]) > 1]
            neg = dedup(neg)

            if len(neg) == 0:
                continue


            d = {'query':query,'pos':pos,'neg':neg}
            fopen.write(f'{json.dumps(d).decode()}\n')
            fopen.flush()
            pointer.index = n
            pointer._save()

In [139]:
# loop((range(10), 0))

In [140]:
import mp

mp.multiprocessing(range(len(data)), loop, cores = 40, returned = False)

 10%|█         | 495/4781 [12:47<2:48:16,  2.36s/it]